In [ ]:
vantage_broker_url = "http://localhost"
vantage_broker_username = "inforing-user"
vantage_broker_password = "inforing-password"

vantage_broker_encryption = None
vantage_broker_port = 5000
vantage_broker_api_path = "/api"

In [ ]:
from vantage6.client import Client
client = Client(vantage_broker_url, vantage_broker_port, vantage_broker_api_path, log_level='critical')
client.authenticate(vantage_broker_username, vantage_broker_password)
client.setup_encryption(vantage_broker_encryption)

In [ ]:
import json
collaboration_list = client.collaboration.list()
# print(json.dumps(collaboration_list, indent=2))
collaboration_index = 0
organization_ids_ = [ ]

for organization in collaboration_list[collaboration_index]['organizations']:
    organization_ids_.append(organization['id'])
# print(json.dumps(client.node.list(), indent=2))

In [ ]:
input_ = {
    "method":"count_citizens",
    "master": True
}

task = client.post_task(
    name="CountCitizens",
    image="ghcr.io/elsa-lab-p-d/wp3-demo/count",
    collaboration_id=collaboration_list[collaboration_index]['id'],#Get the first collaboration associated with user
    input_= input_,
    organization_ids=[organization_ids_[0]],
)

In [ ]:
print(f"run-id: {task}")

In [ ]:
import time
import json
task_info = client.task.get(task['id'])
i=0
while ((task_info['status']!="completed") and i<10):
    sleep_time = 5
    time.sleep(sleep_time)
    print(f"Waiting for {sleep_time} seconds")
    i+=1
    task_info = client.task.get(task['id'])

print(f"Status: {task_info['status']}")
print(f"Result ID: {task_info['results'][0]['id']}")

if (task_info['status']=="completed"):
    results = client.result.get(task_info["results"][0]["id"])
    print(json.dumps(results['result'], indent=2))
else:
    results = client.result.get(task_info["results"][0]["id"])
    # results = client.result.from_task(task_info["run_id"])
    print(results[0]['log'])